# <center>Applying Machine Learning to Sentiment Analysis</center>

# <center>Tổng quan

Trong thời đại internet và phương tiện truyền thông xã hội hiện đại, ý kiến, đánh giá của mọi người, và khuyến nghị đã trở thành một nguồn tài nguyên quý giá cho khoa học chính trị và doanh nghiệp. Những dữ liệu này thường có được dưới dạng văn bản, điều này thúc đẩy sự phát triển của nhánh xử lý ngôn ngữ tự nhiên (NLP) trong Machine Learning. Trong chương này chúng ta sẽ bàn về _Sentiment Analysis_ một nhánh trong NLP và học cách sứ dụng thuật toán ML để phân loại tài liệu dự trên thái độ của người viết. Chúng ta làm việc cùng dataset **IMDb** với 50.000 lượt đánh giá phim và phân biệt thái độ của người viết.

Cơ sở dữ liệu phim Internet (IMDb) đã được Andrew Maas và những người khác thu thập. Bộ dữ liệu đánh giá phim bao gồm 50.000 đánh giá phim được gán nhãn là  tích cực (1) và tiêu cực (-1); Với giả định, tích cực có nghĩa là một bộ phim được xếp hạng có hơn sáu sao trên IMDb và tiêu cực có nghĩa là một bộ phim được xếp hạng có ít hơn năm sao trên IMDb. 

Nội dung bao gồm:
- Xây vectơ đặc trưng từ các tài liệu
- Xây dựng mô hình machine learning để phân loại tích cực hay tiêu cực.
- Sử dụng out-of-core learning
- Giới thiệu về Toppic modeling (LDA)

## 1. Giới thiệu về bag-of-words mode

Trong phần này chúng ta sẽ tìm hiểu về **bag-of-words**, cho phép chúng ta biểu diễn văn bản dưới dạng các vectơ đặc trưng số.
- Ta sẽ tạo một tập các term khác từ corpus.
- Ta sẽ biểu diễn vectơ đặc trưng của mỗi document bằng tần suất xuất hiện của mỗi term trong document đó. Vì mỗi document có số lượng term duy nhất thường rất ít so với các term trong bag-of-words nên vectơ của ta đa phần là số 0, chính vì nên nó còn được gọi là _sparse vector_.

# 2. Xây dựng vectơ đặc trưng

## 2.1  bag-of-word

Để xây dựng vectơ đặc trưng cho các document ta sử dụng `CountVectorizer` trong thư viện `scikit-learn`.

In [1]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
count = CountVectorizer()
docs = np.array(
    ['The sun is shining',
     'The weather is sweet',
     'The sun is shining, the weather is sweet,and one and one is two'])
bag = count.fit_transform(docs)

In [2]:
print(count.vocabulary_)

{'the': 6, 'sun': 4, 'is': 1, 'shining': 3, 'weather': 8, 'sweet': 5, 'and': 0, 'one': 2, 'two': 7}


In [3]:
print(bag.toarray())

[[0 1 0 1 1 0 1 0 0]
 [0 1 0 0 0 1 1 0 1]
 [2 3 2 1 1 1 2 1 1]]


Các giá trị này trong các vectơ đặc trưng được gọi là tần số thuật ngữ thô (**raw term frequencies**).

$tf(t,d)$ Số lần của một term $t$ xuất hiện trong document $d$. Trong mô hình bag-of-words, thứ tự từ trong một câu hoặc tài liệu không quan trọng. Thứ tự xuất hiện các tần số thuật ngữ trong vectơ đặc trưng được lấy từ các chỉ số từ vựng, thường được gán theo thứ tự abc.

## 2.2 TF-IDF

Trong thực tế có những từ xuất hiện trên nhiều document. Những từ này thường không chứa thông tin hữu ích cho việc phân loại nhưng lại có trọng số lớn làm cho mô hình phân loại bị bias. Ở phần này, ta sẽ tìm hiểu về một kỹ thuật hữu ích được gọi là **term frequency-inverse document frequency (tf-idf)**, nó giúp ta giảm trọng số của những từ thường xuyên xuất hiện trong các vectơ đặc trưng. Tf-idf có thể được định nghĩa là thương số của $tf$ và $idf$:
$$tfidf(t,d)=tf(t,d)\times idf(t,d)$$
Với $tf (t, d)$ là tần số term $t$ trên document $d$ và $idf (t, d)$ là tần số tài liệu nghịch đảo, có thể được tính như sau:
$$idf(t,d)=\log\frac{n_d}{1+df(d,t)}$$
Với $n_d$ là tổng số document và $df (d, t)$ là số lượng document $d$ có chứa thuật ngữ $t$.

Trong thư viện ``scikit-learn`` có lớp ``TfidfTransformer``, nó lấy $tf$ từ lớp ``CountVectorizer`` làm đầu vào và biến chúng thành $tfidf$:

In [4]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf = TfidfTransformer(use_idf=True,norm='l2',smooth_idf=True)
np.set_printoptions(precision=2)
print(tfidf.fit_transform(count.fit_transform(docs))
.toarray())

[[0.   0.43 0.   0.56 0.56 0.   0.43 0.   0.  ]
 [0.   0.43 0.   0.   0.   0.56 0.43 0.   0.56]
 [0.5  0.45 0.5  0.19 0.19 0.19 0.3  0.25 0.19]]


Phương trình $idf$ được thực hiện trong ``scikitlearn`` được tính như sau:
$$idf(t,d)=\log\frac{1+n_d}{1+df(d,t)}$$
Tương tự, tf-idf được tính toán trong `scikit-learn` lệch một chút so với phương trình mặc định mà ta đưa ra trước đó:
$$tf-idf(t,d)=tf(t,d)\times (idf(t,d)+1)$$

## 2.3 Làm sạch dữ liệu văn bản

Trong quá trình thu thập, dữ liệu có thể kèm theo các ký tự đặc biệt, cái biểu cảm, các thẻ của html. Điều này nếu không được xử lý thì nó sẽ làm gia tăng số lượng term trong mô hình bag-of-word hoặc có thể làm sai kết quả dự đoán của mô hình.



In [5]:
import pandas as pd
df = pd.read_csv('movie_data.csv', encoding='utf-8')
df.loc[0, 'review'][-50:]

'is seven.<br /><br />Title (Brazil): Not Available'

Để xứ lý nó, ta sử dụng một số kỹ thuật cơ bản trong xử lý chuỗi như áp dụng biểu thức chính quy. Ví dụ như sau, ta định nghĩa hàm `preprocessor` để loại bỏ các ký tự đặc biệt, biểu tượng cảm xúc cũng như các thẻ html:

In [6]:
import re
def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',
    text)
    text = (re.sub('[\W]+', ' ', text.lower()) +
    ' '.join(emoticons).replace('-', ''))
    return text

In [7]:
preprocessor(df.loc[0, 'review'][-50:])

'is seven title brazil not available'

Áp dụng cho toàn bộ dataset của mình:

In [8]:
df['review'] = df['review'].apply(preprocessor)

# 2.4 Xử lý tài liệu thành các `token`

Để thực hiện các kỹ thuật như bag-of-word hay tf-idf ta cần xử lý các document thành cách term riêng lẻ. Các đơn giản nhất là ta tách theo khoảng trắng.

In [9]:
def tokenizer(text):
    return text.split()
tokenizer('runners like running and thus they run')

['runners', 'like', 'running', 'and', 'thus', 'they', 'run']

Như ta thấy ở ví dụ trên term ``running`` có cùng nghĩa với term ``run`` nhưng lại được chia thành 2 term khác nhau. Điều này làm cho bag-of-word của ta nhiều hơn nhưng lại không mang ý nghĩa. Để tránh điều này, ta cần chuyển các term về dạng gốc của nó. Một kỹ thuật phổ biến là stemming, bộ công cụ xử lý ngôn ngữ tự nhiên Natural Language Toolkit (**NLTK**) cung cấp cho ta lớp `PorterStemmer` hoặc `LancasterStemmer` để thực hiện điều này.

In [10]:
from nltk.stem import PorterStemmer
porter = PorterStemmer()
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]
tokenizer_porter('runners like running and thus they run')

['runner', 'like', 'run', 'and', 'thu', 'they', 'run']

In [11]:
from nltk.stem import LancasterStemmer
lancaster=LancasterStemmer()
def tokenizer_lacaster(text):
    return [lancaster.stem(word) for word in text.split()]
tokenizer_lacaster('runners like running and thus they run')

['run', 'lik', 'run', 'and', 'thu', 'they', 'run']

In [12]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

# 3. Xây dựng mô hình machine learning để phân loại

Chúng ta sử dụng mô hình **logistic regression** để phân loại các lượt đánh giá là tích cực hay tiêu cực. Ta sẽ chia ``DataFrame`` của các đoạn đánh giá đã được làm sạch thành 25.000 tài liệu để đào tạo và 25.000 tài liệu để thử nghiệm:

In [13]:
X_train = df.loc[:25000, 'review'].values
y_train = df.loc[:25000, 'sentiment'].values
X_test = df.loc[25000:, 'review'].values
y_test = df.loc[25000:, 'sentiment'].values

Ta sử dụng ``GridSearchCV`` để tìm bộ ``hyperparameter`` tốt nhất:

In [14]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(strip_accents=None,lowercase=False,preprocessor=None)
param_grid = [{'vect__ngram_range': [(1,1)],'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer,tokenizer_porter,tokenizer_lacaster],
                'clf__penalty': ['l1', 'l2'],
                'clf__C': [1.0, 10.0, 100.0]
            },
            {'vect__ngram_range': [(1,1)],
                'vect__stop_words': [stop, None],
                'vect__tokenizer': [tokenizer,tokenizer_porter,tokenizer_lacaster],
                'vect__use_idf':[False],
                'vect__norm':[None],
                'clf__penalty': ['l1', 'l2'],
                'clf__C': [1.0, 10.0, 100.0]}
            ]
lr_tfidf = Pipeline([('vect', tfidf),('clf',LogisticRegression(random_state=0,solver='liblinear'))])
gs_lr_tfidf = GridSearchCV(lr_tfidf, param_grid,scoring='accuracy',cv=5, verbose=2,n_jobs=1)
gs_lr_tfidf.fit(X_train, y_train)

Fitting 5 folds for each of 72 candidates, totalling 360 fits
[CV] clf__C=1.0, clf__penalty=l1, vect__ngram_range=(1, 1), vect__stop_words=['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  clf__C=1.0, clf__penalty=l1, vect__ngram_range=(1, 1), vect__stop_words=['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'othe

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    6.2s remaining:    0.0s


[CV]  clf__C=1.0, clf__penalty=l1, vect__ngram_range=(1, 1), vect__stop_words=['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'othe

...

[CV]  clf__C=100.0, clf__penalty=l2, vect__ngram_range=(1, 1), vect__norm=None, vect__stop_words=['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'm

[CV]  clf__C=100.0, clf__penalty=l2, vect__ngram_range=(1, 1), vect__norm=None, vect__stop_words=['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'm

[CV]  clf__C=100.0, clf__penalty=l2, vect__ngram_range=(1, 1), vect__norm=None, vect__stop_words=None, vect__tokenizer=<function tokenizer at 0x0000023E2FE8DEA0>, vect__use_idf=False, total=  19.5s
[CV] clf__C=100.0, clf__penalty=l2, vect__ngram_range=(1, 1), vect__norm=None, vect__stop_words=None, vect__tokenizer=<function tokenizer at 0x0000023E2FE8DEA0>, vect__use_idf=False 
[CV]  clf__C=100.0, clf__penalty=l2, vect__ngram_range=(1, 1), vect__norm=None, vect__stop_words=None, vect__tokenizer=<function tokenizer at 0x0000023E2FE8DEA0>, vect__use_idf=False, total=  14.6s
[CV] clf__C=100.0, clf__penalty=l2, vect__ngram_range=(1, 1), vect__norm=None, vect__stop_words=None, vect__tokenizer=<function tokenizer at 0x0000023E2FE8DEA0>, vect__use_idf=False 
[CV]  clf__C=100.0, clf__penalty=l2, vect__ngram_range=(1, 1), vect__norm=None, vect__stop_words=None, vect__tokenizer=<function tokenizer at 0x0000023E2FE8DEA0>, vect__use_idf=False, total=  16.7s
[CV] clf__C=100.0, clf__penalty=l2, vect

[Parallel(n_jobs=1)]: Done 360 out of 360 | elapsed: 562.1min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=False,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                                        n

Sau khi quá trình tìm kiếm kết thúc, ta có thể xem bộ tham số tốt nhất:

In [15]:
print('Best parameter set: %s ' % gs_lr_tfidf.best_params_)

Best parameter set: {'clf__C': 10.0, 'clf__penalty': 'l2', 'vect__ngram_range': (1, 1), 'vect__stop_words': None, 'vect__tokenizer': <function tokenizer at 0x0000023E2FE8DEA0>} 


In [16]:
print('CV Accuracy: %.3f',gs_lr_tfidf.best_score_)

CV Accuracy: %.3f 0.8970842631473704


In [17]:
clf = gs_lr_tfidf.best_estimator_
print('Test Accuracy: %.3f',clf.score(X_test, y_test))

Test Accuracy: %.3f 0.89876


# 4. Sử dụng out-of-core learning

Như ta thấy thì việc tính toán tạo vectơ đặc trưng của 50 000 lượt đánh giá phim trong `GridSearchCV` sẽ rất tốn kém. Trong thực tế có nhiều ứng dụng với dữ liệu rất lớn vượt quá bộ nhớ của máy tính. Để giải quyết vấn đề đó thì ta sẽ được tiếp cận với kỹ thuật out-of-core learning, cho phép ta làm việc với các bộ dữ liệu lớn như vậy bằng cách train trên các batch dữ liệu nhỏ hơn. Ta sẽ sử dụng chức năng `partial_fit` của `SGDClassifier` trong `scikit-learn` để truyền phát tài liệu trực tiếp từ ổ đĩa cục bộ và huấn luyện mô hình **logistic regression** bằng cách sử dụng các batch tài liệu nhỏ.

Ta định nghĩa lại hàm ``tokenizer`` để tiền xử lý dữ liệu khi đọc lên trực tiếp từ ở đĩa.

In [1]:
import numpy as np
import re
from nltk.corpus import stopwords
stop = stopwords.words('english')
def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',
    text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) \
    + ' '.join(emoticons).replace('-', '')
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized

Tiếp theo ta định nghĩa hàm đọc và trả về một điểm dữ liệu tại một thời điểm:

In [2]:
def stream_docs(path):
    with open(path, 'r', encoding='utf-8') as csv:
        next(csv) # skip header
        for line in csv:
            text, label = line[:-3], int(line[-2])
            yield text, label
next(stream_docs(path='movie_data.csv'))

('"In 1974, the teenager Martha Moxley (Maggie Grace) moves to the high-class area of Belle Haven, Greenwich, Connecticut. On the Mischief Night, eve of Halloween, she was murdered in the backyard of her house and her murder remained unsolved. Twenty-two years later, the writer Mark Fuhrman (Christopher Meloni), who is a former LA detective that has fallen in disgrace for perjury in O.J. Simpson trial and moved to Idaho, decides to investigate the case with his partner Stephen Weeks (Andrew Mitchell) with the purpose of writing a book. The locals squirm and do not welcome them, but with the support of the retired detective Steve Carroll (Robert Forster) that was in charge of the investigation in the 70\'s, they discover the criminal and a net of power and money to cover the murder.<br /><br />""Murder in Greenwich"" is a good TV movie, with the true story of a murder of a fifteen years old girl that was committed by a wealthy teenager whose mother was a Kennedy. The powerful and rich f

Bây giờ chúng ta sẽ định nghĩa một hàm `get_minibatch`, sẽ lấy một luồng tài liệu từ hàm `stream_docs` và trả về một số lượng document cụ thể được chỉ định bởi tham số `size`:

In [3]:
def get_minibatch(doc_stream, size):
    docs, y = [], []
    try:
        for _ in range(size):
            text, label = next(doc_stream)
            docs.append(text)
            y.append(label)
    except StopIteration:
        return None, None
    return docs, y

Ta không thể sử dụng `CountVectorizer` hoặc `TfidfVectorizer` vì nó cần giữ tất cả các vectơ đặc trưng của tập dữ liệu huấn luyện trong bộ nhớ. Tuy nhiên, một `vectorizer` hữu ích khác để xử lý văn bản được triển khai trong `scikit-learn` là `HashingVectorizer`. `HashingVectorizer` độc lập với dữ liệu và sử dụng thủ thuật băm thông qua chức năng MurmurHash3.

In [4]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier
vect = HashingVectorizer(decode_error='ignore',n_features=2**21,preprocessor=None,tokenizer=tokenizer)
clf = SGDClassifier(loss='log', random_state=1)
doc_stream = stream_docs(path='movie_data.csv')

In [25]:
import pyprind
pbar = pyprind.ProgBar(49)
classes = np.array([0, 1])
for _ in range(49):
    X_train, y_train = get_minibatch(doc_stream, size=1000)
    if not X_train:
        break
    X_train = vect.transform(X_train)
    clf.partial_fit(X_train, y_train, classes=classes)
    pbar.update()

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:36


Sau khi hoàn thành quá trình trainning, ta sẽ sử dụng 5.000 tài liệu cuối cùng để đánh giá hiệu suất của mô hình:

In [26]:
X_test, y_test = get_minibatch(doc_stream, size=1000)
X_test = vect.transform(X_test)
print('Accuracy: %.3f' % clf.score(X_test, y_test))

Accuracy: 0.868


# 5. Topic modeling với Latent Dirichlet Allocation (LDA)

Topic modeling thực hiện nhiệm vụ gán nhãn chủ đề cho document chưa được gán nhãn. Ví dụ, một ứng dụng điển hình sẽ là phân loại tài liệu trong một văn bản lớn của các bài báo. Trong các ứng dụng của mô hình chủ đề, sau đó nhắm đến việc gán nhãn danh mục cho các bài viết đó, ví dụ: thể thao, tài chính, tin tức thế giới, chính trị, tin tức địa phương, v.v. 

Có thể coi mô hình hóa chủ đề như một nhiệm vụ phân cụm, một thể loại con của học tập không giám sát. Một kỹ thuật phổ biến để mô hình hóa chủ đề được gọi là **Latent Dirichlet Allocation (LDA)**.

## 5.1 Phân loại document với LDA

Vì **LDA** đòi hỏi kiến thức về toán học và hiểu biết về xác xuất Bayesian nên chúng ta sẽ tiếp cận thuật toán này dựa trên thực hành.

**LDA** là một mô hình xác suất tổng quát cố gắng tìm các nhóm từ xuất hiện thường xuyên cùng nhau trên các document khác nhau. Những từ thường xuyên xuất hiện này đại diện cho các chủ đề. Input của **LDA** là mô hình bag-of-word, **LAD** dưa ra một ma trận bag-of-word làm đầu vào, LDA phân tách nó thành hai ma trận mới:
- A document-to-topic matrix
- A word-to-topic matrix

LDA phân tách ma trận bag-of-words thành hai ma trận nên khi nhân hai ma trận này lại ta sẽ thu được ma trận đầu vào. Trong thực tế, ta quan tâm đến những chủ đề mà LDA tìm thấy trong ma trận bag-of-words. Nhược điểm duy nhất có thể là ta phải xác định trước số lượng chủ đề.

# 5.2 LDA với scikit-learn

In [27]:
import pandas as pd
df = pd.read_csv('movie_data.csv', encoding='utf-8')

In [28]:
from sklearn.feature_extraction.text import CountVectorizer
count = CountVectorizer(stop_words='english',max_df=.1,max_features=5000)
X = count.fit_transform(df['review'].values)

In [29]:
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components=10,random_state=123,learning_method='batch')
X_topics = lda.fit_transform(X)

In [30]:
lda.components_.shape

(10, 5000)

In [31]:
n_top_words = 5
feature_names = count.get_feature_names()
for topic_idx, topic in enumerate(lda.components_):
    print("Topic %d:" % (topic_idx + 1))
    print(" ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))

Topic 1:
worst minutes awful script stupid
Topic 2:
family mother father children girl
Topic 3:
american war dvd music tv
Topic 4:
human audience cinema art sense
Topic 5:
police guy car dead murder
Topic 6:
horror house sex girl woman
Topic 7:
role performance comedy actor performances
Topic 8:
series episode war episodes tv
Topic 9:
book version original read novel
Topic 10:
action fight guy guys cool


In [32]:
horror = X_topics[:, 5].argsort()[::-1]
for iter_idx, movie_idx in enumerate(horror[:3]):
    print('\nHorror movie #%d:' % (iter_idx + 1))
    print(df['review'][movie_idx][:300], '...')


Horror movie #1:
House of Dracula works from the same basic premise as House of Frankenstein from the year before; namely that Universal's three most famous monsters; Dracula, Frankenstein's Monster and The Wolf Man are appearing in the movie together. Naturally, the film is rather messy therefore, but the fact that ...

Horror movie #2:
Okay, what the hell kind of TRASH have I been watching now? "The Witches' Mountain" has got to be one of the most incoherent and insane Spanish exploitation flicks ever and yet, at the same time, it's also strangely compelling. There's absolutely nothing that makes sense here and I even doubt there  ...

Horror movie #3:
<br /><br />Horror movie time, Japanese style. Uzumaki/Spiral was a total freakfest from start to finish. A fun freakfest at that, but at times it was a tad too reliant on kitsch rather than the horror. The story is difficult to summarize succinctly: a carefree, normal teenage girl starts coming fac ...
